### 출력데이터 초과오류
- jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10  

### 오류코드: (1205) Lock wait timeout exceeded  => CMD 설정
1. show variables like '%wait_timeout%';
2. set innodb_lock_wait_timeout= 28800;

# Connection and Cursor

### AWS 추가

In [3]:
# import pymysql
# conn = pymysql.connect(host='jkm1.cdoztymfmdwr.ap-northeast-2.rds.amazonaws.com',
#                                      database='ship',
#                                      user='admin',
#                                      password='administrator')

### 민경 DB 추가

In [ ]:
# import pymysql
# conn = pymysql.connect(host='10.125.121.204',
#                                      database='ship',
#                                      user='root',
#                                      password='tiger')

#### 1. arrivalPort
- 수기입력

#### 2. schedules 테이블 입력(실제데이터)
- num
- ship_shipId
- arrivalPort_arrivalName
- departure
- departTime

In [70]:
import random
import pandas as pd

## 선박 MMSI 리스트
mmsi = pd.read_csv('C:/Users/user/Desktop/항만운영정보시스템data/ModelingDataset/portmis.csv', encoding = 'cp949', usecols = [3])
shiplist = mmsi.loc[:,'mmsi'].to_list()
    
## 도착항 리스트
p = "인천항"

## 출발항 리스트
departPort = pd.read_csv('C:/Users/user/Desktop/항만운영정보시스템data/ModelingDataset/portmis.csv', encoding = 'cp949', usecols = [11])
depart = departPort.loc[:,'전출항지'].to_list()

## 출발시각 리스트(더미)
departT = []

for i in range(0, len(shiplist)) :
    mm = str(random.randint(1, 12))
    h = str(random.randint(0, 23))
    m = str(random.randint(0, 59))
    s = str(random.randint(0, 59))
    if mm == '2':
        d = str(random.randint(1, 28))
    elif mm == '4' or '6' or '9' or '11':
        d = str(random.randint(1, 30))
    else:
        d = str(random.randint(1, 31))
    if len(mm) != 2 :
        mm = "0"+mm
    if len(d) != 2 :
        d = "0"+d
    if len(h) != 2 :
        h = "0"+h
    if len(m) != 2 :
        m = "0"+m
    if len(s) != 2 :
        s = "0"+s
    t = "2022"+ "-" + mm + "-"+ d +" "+h + ":" + m + ":" + s
    departT.append(t)

In [71]:
list_schedulesData = []
for i in range(1, len(shiplist)) :
    a = i,shiplist[i] ,p,random.choice(depart),random.choice(departT)
    list_schedulesData.append(a)

In [72]:
list_schedulesData

[(1, 477301900, '인천항', 'HONG KONG', '2022-02-11 10:50:03'),
 (2, 441387000, '인천항', 'ZHANGJIAGANG', '2022-07-19 07:30:03'),
 (3, 440026590, '인천항', '평택.당진', '2022-06-10 19:38:00'),
 (4, 440403850, '인천항', '기타항', '2022-09-30 06:28:27'),
 (5, 440148840, '인천항', 'YANTAI APT', '2022-12-23 17:16:30'),
 (6, 151810101, '인천항', 'XIAMEN', '2022-12-07 03:39:09'),
 (7, 232024773, '인천항', 'TIANJIN', '2022-10-20 05:48:17'),
 (8, 371369000, '인천항', 'YANTAI APT', '2022-03-16 11:18:56'),
 (9, 353727000, '인천항', '군산', '2022-10-12 09:27:10'),
 (10, 440162170, '인천항', 'LAEM CHABANG', '2022-07-08 08:29:48'),
 (11, 215009000, '인천항', '기타항', '2022-08-02 04:57:18'),
 (12, 898520089, '인천항', '기타항', '2022-09-10 04:36:20'),
 (13, 563955000, '인천항', 'HUANGPU PT', '2022-10-13 18:05:55'),
 (14, 538008215, '인천항', '부산', '2022-01-01 06:02:23'),
 (15, 441768000, '인천항', '기타항', '2022-03-15 07:07:26'),
 (16, 17670152, '인천항', 'SAKAISENBOKU', '2022-03-06 02:00:09'),
 (17, 440714179, '인천항', 'OSAKA', '2022-10-13 05:00:59'),
 (18, 212546

In [76]:
# 민경 db
# import pymysql
# conn = pymysql.connect(host='10.125.121.204',
#                                      database='ship',
#                                      user='musthave',
#                                      password='tiger')

# #AWS db
# conn = pymysql.connect(host='jkm1.cdoztymfmdwr.ap-northeast-2.rds.amazonaws.com',
#                                      database='ship',
#                                      user='admin',
#                                      password='administrator')

try:
    with conn.cursor() as cursor:
        sql = 'INSERT INTO schedules (num, ship_shipId, arrivalPort_arrivalName, departure, departTime) VALUES (%s, %s, %s, %s, %s)'
        cursor.executemany(sql, list_schedulesData)
    conn.commit()
finally:
    conn.close()

In [40]:
len(list_schedulesData)

395

#### 3. ship 테이블
- shipId
- shipCode 
- shipName
- shipUse

In [46]:
## 선박 정보
ship = pd.read_csv('C:\/Users/user/Desktop/항만운영정보시스템data/shipDummy.csv', encoding = 'cp949')
ship.head()

,shipCode,shipId,shipName,shipUse
0,146805,440413160,비룡호,급유선
1,VROF4,477301900,SITC HEBEI,풀컨테이너선
2,D7WG,441387000,HECAN,풀컨테이너선
3,161822,440026590,9민성호,석유제품 운반선
4,130020,440403850,해령호,석유제품 운반선


In [47]:
ship = ship.drop_duplicates(subset='shipId')

In [48]:
shipCode = ship.loc[:,'shipCode'].to_list()
shipId = ship.loc[:,'shipId'].to_list()
shipName =ship.loc[:,'shipName'].to_list()
shipUse = ship.loc[:,'shipUse'].to_list()

In [49]:
list_shipData = []
for i in range(len(shipCode)) :
    a = shipId[i],shipCode[i], shipName[i], shipUse[i]
    list_shipData.append(a)

In [50]:
list_shipData

[(440413160, '146805', '비룡호', '급유선'),
 (477301900, 'VROF4', 'SITC HEBEI', '풀컨테이너선'),
 (441387000, 'D7WG', 'HECAN', '풀컨테이너선'),
 (440026590, '161822', '9민성호', '석유제품 운반선'),
 (440403850, '130020', '해령호', '석유제품 운반선'),
 (440148840, '114806', '디비써니', '케미칼 운반선'),
 (151810101, '151810', '아름호', '급유선'),
 (232024773, 'MGGA6', 'HYUNDAI MARS', '풀컨테이너선'),
 (371369000, '3FDX9', 'POS BANGKOK', '풀컨테이너선'),
 (353727000, '3EGC3', 'LILAC PROMENADE', 'LPG 운반선'),
 (440162170, '210009', '대복가스호', 'LPG 운반선'),
 (215009000, '9HA4916', 'TRIESTE TRADER', '풀컨테이너선'),
 (898520089, '130081', '은성호', '석유제품 운반선'),
 (563955000, '9VIB6', 'KOTA TAMPAN', '풀컨테이너선'),
 (538008215, 'V7A2092', 'OLYMPIC LIFE', '원유운반선'),
 (441768000, 'D7SO', 'SK SUPREME', 'LNG 운반선'),
 (17670152, '176701', '옹진페리호', '일반화물선'),
 (440714179, 'BATD', 'MING YUE', '세미(혼재)컨테이너선'),
 (212546000, '5BQV4', 'SENECA', '산물선(벌크선)'),
 (477466300, 'VRNQ7', 'SITC SHANDONG', '풀컨테이너선'),
 (312664000, 'V3CY3', 'PU REN', '일반화물선'),
 (440121610, '80186', '7민성호', '석유제품 운반선'),
 

In [77]:
# 민경 db
# import pymysql
# conn = pymysql.connect(host='10.125.121.204',
#                                      database='ship',
#                                      user='musthave',
#                                      password='tiger')

# #AWS db
# conn = pymysql.connect(host='jkm1.cdoztymfmdwr.ap-northeast-2.rds.amazonaws.com',
#                                      database='ship',
#                                      user='admin',
#                                      password='administrator')

try:
    with conn.cursor() as cursor:
        sql = 'INSERT INTO ship (shipId, shipCode, shipName, shipUse) VALUES (%s, %s, %s, %s)'
        cursor.executemany(sql, list_shipData)
    conn.commit()
finally:
    conn.close()

In [38]:
len(list_shipData)

283

#### 4. shiplog 테이블 더미생성 (구현중)
- num
- shipId
- insertTime
- shipLat
- shipLon
- speed
- arrivalTime
- takeTime
- accuracy

In [7]:
import datetime

# 실제 도착시간 (port-mis에서 추출, 22.12.15 ~ 22.12.28)
arrT = pd.read_csv('C:/Users/user/Desktop/항만운영정보시스템data/ModelingDataset/portmis.csv', encoding = 'cp949', usecols = [9])
arrivalT = arrT.loc[:,'입항일시'].to_list()

## 선박 현재 위도
shiplat = []

## 선박 현재 경도
shiplon = []

## 선박 속도
speed = []

## 소요시간
takeTime = []

## 정확도
accuracy = []

# 위도, 경도: 랜덤 시작 + 랜덤a. (a의 범위는 시속이 빨라지면 많이증가, 시속이 느려지면 적게 증가)
for i in range(len(shiplist)) :
    for j in len(current):
        current =  datetime.datetime.now()
        one_minute_later = current + datetime.timedelta(minutes=1)

        # 가장 느린 편에 속하는 원유운반선(탱커/Tanker)과 살물선(벌커/Bulker)의 속도는 15~16노트(knot/시속 28∼30km)
        # 컨테이너선이나 자동차운반선은 평균 24~26노트(시속 44∼48km)
        while ((speed[i]) * (1/60.0)) < 80.0: 
            speed.append(round(random.uniform(0, 48),2))  
            if speed[j] > speed [j-1]:
                shiplat[i+1]  > shiplat[i]
                shiplon[i+1]  > shiplon[i]
            elif speed[i] == speed [i-1]:
                shiplat[i+1] == shiplat[i]
                shiplon[i+1] == shiplon[i]
            else:
                shiplat[i+1] < shiplat[i]
                shiplon[i+1] < shiplon[i]

                # 위도 37도에서는 위도 1도 변할 때 약 110.9782 km 차이가 난다. 때문에 인천항 주변 80km범위는 위도 차가 약0.721도 난다.
                shiplat.append(round(random.uniform(36.8, 37.521), 6) +  round(random.uniform(0, 0.05), 6))
                # 위도 37도에서는 경도 1도 변할 때 약 93.6472km 차이가 난다. 때문에 인천항 주변 80km 범위는 경도 차가 약0.85428도 난다.
                shiplon.append(round(random.uniform(125.6, 126.45428),6) + round(random.uniform(0, 0.042),6))

            break
        
# now = str(datetime.now())

# realtime = now[:-7]

# ## 도착시각
# arrT = []

# for i in range(0, len(shiplist)) :
#     d = str(random.randint(15, 28))
#     h = str(random.randint(0, 23))
#     m = str(random.randint(0, 59))
#     s = str(random.randint(0, 59))
#     if len(h) != 2 :
#         h = "0"+h
#     if len(m) != 2 :
#         m = "0"+m
#     if len(s) != 2 :
#         s = "0"+s
#     t = "2023-12-"+ d + " "+ h + ":" + m + ":" + s
#     arrT.append(t)

NameError: name 'current' is not defined

#### 5. weather 테이블

- obsId
- time
- temp
- pressure
- windDirec
- windSpeed
- obsLat
- obsLon
- tideLevel

In [5]:
import pandas as pd
import pymysql

In [86]:
## 날씨정보
weather = pd.read_csv('C:\/Users/user/Desktop/항만운영정보시스템data/ModelingDataset/weather.csv', encoding = 'cp949', index_col = 0)
weather.reset_index(drop = True)

,관측소명,관측시간,위도,경도,조위(cm),풍속(m/s),풍향(deg),기온(℃),기압(hPa)
0,인천송도,2022-12-15 14:49,37.338056,126.586111,459,5.2,136,-5.4,1026.6
1,인천송도,2022-12-15 0:01,37.338056,126.586111,456,4.3,140,-5.5,1026.7
2,인천송도,2022-12-15 0:02,37.338056,126.586111,455,5.3,132,-5.5,1026.7
3,인천송도,2022-12-15 0:03,37.338056,126.586111,453,5.5,126,-5.5,1026.7
4,인천송도,2022-12-15 0:04,37.338056,126.586111,451,4.3,136,-5.5,1026.8
...,...,...,...,...,...,...,...,...,...
138539,덕적도,2022-12-28 23:55,37.226300,126.156600,419,1.4,304,-3.3,1033.0
138540,덕적도,2022-12-28 23:56,37.226300,126.156600,417,1.7,298,-3.3,1033.0
138541,덕적도,2022-12-28 23:57,37.226300,126.156600,414,1.7,304,-3.3,1033.0
138542,덕적도,2022-12-28 23:58,37.226300,126.156600,412,1.6,301,-3.4,1033.0


In [89]:
# 컬럼 리스트 생성
obsId = weather.iloc[:,0].to_list()
time = weather.iloc[:,1].to_list()
temp = weather.iloc[:,7].to_list()
pressure = weather.iloc[:,8].to_list()
windDirec= weather.iloc[:,6].to_list()
windSpeed = weather.iloc[:,5].to_list()
obsLat=weather.iloc[:,2].to_list()
obsLon =weather.iloc[:,3].to_list()
tideLevel = weather.iloc[:,4].to_list()

In [90]:
obsLat[-1]

37.2263

In [91]:
list_sqlData = []
# sql문 생성
for i in range(len(weather)) :
    a = obsId[i], time[i] ,temp[i],pressure[i],windDirec[i],windSpeed[i], round(obsLat[i],6),round(obsLon[i],6),tideLevel[i]
    list_sqlData.append(a)

In [92]:
list_sqlData

[('인천송도',
  '2022-12-15 14:49',
  -5.4,
  1026.6,
  136,
  5.2,
  37.338056,
  126.586111,
  459),
 ('인천송도',
  '2022-12-15 0:01',
  -5.5,
  1026.7,
  140,
  4.3,
  37.338056,
  126.586111,
  456),
 ('인천송도',
  '2022-12-15 0:02',
  -5.5,
  1026.7,
  132,
  5.3,
  37.338056,
  126.586111,
  455),
 ('인천송도',
  '2022-12-15 0:03',
  -5.5,
  1026.7,
  126,
  5.5,
  37.338056,
  126.586111,
  453),
 ('인천송도',
  '2022-12-15 0:04',
  -5.5,
  1026.8,
  136,
  4.3,
  37.338056,
  126.586111,
  451),
 ('인천송도',
  '2022-12-15 0:05',
  -5.5,
  1026.7,
  138,
  4.3,
  37.338056,
  126.586111,
  449),
 ('인천송도',
  '2022-12-15 0:06',
  -5.6,
  1026.7,
  139,
  4.9,
  37.338056,
  126.586111,
  447),
 ('인천송도',
  '2022-12-15 0:07',
  -5.6,
  1026.8,
  142,
  4.3,
  37.338056,
  126.586111,
  446),
 ('인천송도',
  '2022-12-15 0:08',
  -5.5,
  1026.7,
  140,
  4.4,
  37.338056,
  126.586111,
  444),
 ('인천송도',
  '2022-12-15 0:09',
  -5.5,
  1026.7,
  148,
  4.1,
  37.338056,
  126.586111,
  442),
 ('인천송도',
  '2022-1

In [95]:
# Connection and Cursor

# 민경 db
# import pymysql
# conn = pymysql.connect(host='10.125.121.204',
#                                      database='ship',
#                                      user='musthave',
#                                      password='tiger')

# AWS db
conn = pymysql.connect(host='jkm1.cdoztymfmdwr.ap-northeast-2.rds.amazonaws.com',
                                     database='ship',
                                     user='admin',
                                     password='administrator')
try:
    with conn.cursor() as cursor:
        sql = 'INSERT INTO weather (obsId, time, temp, pressure, windDirec, windSpeed, obsLat, obsLon, tideLevel) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'
        cursor.executemany(sql, list_sqlData)
    conn.commit()
finally:
    conn.close()